In [123]:
# add keyword to end of text for train and test data
keyword_list_train = list(clean_train_data['keyword'])
keyword_list_test = list(clean_test_data['keyword'])
key_list_train = ['' if (type(i) == float and np.isnan(i)) else i for i in keyword_list_train]
key_list_test = ['' if (type(i) == float and np.isnan(i)) else i for i in keyword_list_test]
text_list_train = list(clean_train_data['text'])
text_list_test = list(clean_test_data['text'])

# iterate through and add to end, with space in-between
for i in range(len(clean_train_data['keyword'])):
    text_list_train[i] = text_list_train[i] + ' ' + key_list_train[i]

for i in range(len(clean_test_data['keyword'])):
    text_list_test[i] = text_list_test[i] + ' ' + key_list_test[i]

In [124]:
# relabel text
clean_train_data['text'] = text_list_train
clean_test_data['text'] = text_list_test

In [125]:
# drop keyword col
clean_train_data = clean_train_data.drop(columns='keyword')
clean_test_data = clean_test_data.drop(columns='keyword')

In [126]:
# output as check
clean_train_data.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [222]:
# this checks if the probability of disaster is above 0.5. If so, we label 1.
test_pred_bool = test_pred.copy().astype(int)
for index in range(len(test_pred)): 
    if test_pred[index]>0.5:
        test_pred_bool[index]=1
    else:
        test_pred_bool[index]=0

AttributeError: 'TFSequenceClassifierOutput' object has no attribute 'astype'

In [ ]:
final_predictions = test_pred_bool.flatten()

In [ ]:
accuracy_score(y_test, final_predictions)

---

In [61]:
class TweetClassifier(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        
        super(TweetClassifier, self).__init__(name=name)
        
        bias_initializer = tf.keras.initializers.HeNormal()
        
        self.embedding = Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = GlobalMaxPool1D()
        
        self.dense_1 = Dense(units=dnn_units, activation="relu")
        self.dropout = Dropout(rate=dropout_rate)

        self.last_dense = Dense(units=1, activation="sigmoid")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [67]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [68]:
text_model = TweetClassifier(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [69]:
text_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = text_model.fit(X_train, y_train, epochs=NB_EPOCHS, batch_size=100, validation_data=(X_test, y_test))
text_model.summary()

Epoch 1/5
53/53 [==============================] - 5s 75ms/step - loss: 0.0000e+00 - accuracy: 0.5665 - val_loss: 0.0000e+00 - val_accuracy: 0.5793
Epoch 2/5
53/53 [==============================] - 4s 77ms/step - loss: 0.0000e+00 - accuracy: 0.5671 - val_loss: 0.0000e+00 - val_accuracy: 0.5793
Epoch 3/5
53/53 [==============================] - 4s 76ms/step - loss: 0.0000e+00 - accuracy: 0.5670 - val_loss: 0.0000e+00 - val_accuracy: 0.5793
Epoch 4/5
53/53 [==============================] - 4s 81ms/step - loss: 0.0000e+00 - accuracy: 0.5688 - val_loss: 0.0000e+00 - val_accuracy: 0.5793
Epoch 5/5
53/53 [==============================] - 4s 85ms/step - loss: 0.0000e+00 - accuracy: 0.5579 - val_loss: 0.0000e+00 - val_accuracy: 0.5793
Model: "text_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  6104400   
__________________________________________

In [56]:
results = text_model.evaluate(X_test)
print(results)

71/71 [==============================] - 0s 660us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
[0.0, 0.0]


#### BERT Modelling first attempt

In [173]:
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1', trainable=True)

In [174]:
def build_model(bert_layer, max_seq_len, lr):
        
    input_word_ids = Input(shape=(max_seq_len,), dtype=tf.int32, name='input_word_ids')
    input_mask = Input(shape=(max_seq_len,), dtype=tf.int32, name='input_mask')
    segment_ids = Input(shape=(max_seq_len,), dtype=tf.int32, name='segment_ids')    

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])   
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    optimizer = SGD(learning_rate=lr, momentum=0.8)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [ ]:
all_tokens = []
all_masks = []
all_segments = []

for text in texts:
    text = self.tokenizer.tokenize(text)
    text = text[:self.max_seq_length - 2]
    input_sequence = ['[CLS]'] + text + ['[SEP]']
    pad_len = self.max_seq_length - len(input_sequence)

    tokens = self.tokenizer.convert_tokens_to_ids(input_sequence)
    tokens += [0] * pad_len
    pad_masks = [1] * len(input_sequence) + [0] * pad_len
    segment_ids = [0] * self.max_seq_length

    all_tokens.append(tokens)
    all_masks.append(pad_masks)
    all_segments.append(segment_ids)

return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
X_trn_encoded = self.encode(X.loc[trn_idx, 'text_cleaned'].str.lower())
y_trn = X.loc[trn_idx, 'target_relabeled']
X_val_encoded = self.encode(X.loc[val_idx, 'text_cleaned'].str.lower())
y_val = X.loc[val_idx, 'target_relabeled']

In [181]:
# Model
model = build_model(bert_layer, max_seq_len=128, lr=0.0001)        
model.fit(X_trn_encoded, y_trn, validation_data=(X_val_encoded, y_val), callbacks=[metrics], epochs=10, batch_size=32)

NameError: name 'X_trn_encoded' is not defined

In [156]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_layer(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * ['this is such an amazing movie!']
    * False
    * None
  Keyword arguments: {}

Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * {'input_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/input_mask'), 'input_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/input_type_ids'), 'input_word_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/input_word_ids')}
    * False
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * {'input_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'), 'input_word_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids'), 'input_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_type_ids')}
    * True
    * None
  Keyword a

#### Additional Machine Learning Tests

In [181]:
# testing a support vector machine

# because I haven't done any NLP, I will just use random data with my labels.

y_true = clean_train_data['target']
x1 = np.random.random_sample(len(y_true))
x2 = np.random.random_sample(len(y_true))
xs = list(zip(x1,x2))

In [182]:
X_train, X_test, y_train, y_test = train_test_split(xs, y_true, test_size=0.3, random_state=42)

In [183]:
clf = SVC(probability=True)
clf.fit(X_train, y_train)

SVC(probability=True)

In [184]:
y_pred = clf.predict(X_test)
y_probas = clf.predict_proba(X_test)